In [26]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.dates as mdates
import pandas as pd
import pandas_datareader.data as web
from sklearn import preprocessing
import random
import statistics
import itertools
import numpy as np

In [27]:
df = pd.read_csv('Petrobras.csv')
df = df.drop(columns= 'Titulo')

In [51]:
df

,Data,Sentimento
0,2020-03-13,PO
1,2020-03-13,NE
2,2020-03-13,NG
3,2020-03-12,NG
4,2020-03-12,NE
...,...,...
884,2018-10-19,NE
885,2018-10-18,NG
886,2018-10-17,NG
887,2018-10-17,NG


In [52]:
start = dt.datetime(2019,6,1)
end = dt.datetime(2019,6,10)

df_acoes = web.DataReader('PETR4.SA','yahoo',start,end)

In [53]:
Data = df['Data']

In [54]:
Sentimento = df['Sentimento']

In [55]:
new_dict = (df.groupby('Data')
              .apply(lambda x: [list(y) for y in zip(x['Sentimento'])])
              .to_dict())

In [56]:
Sentimento_Geral = []
for i in range(len(df_acoes.index)):
    if str(df_acoes.index[i])[0:10] in new_dict.keys():
        Sentimento_Geral.append(new_dict[str(df_acoes.index[i])[0:10]])
    else:
        Sentimento_Geral.append('NE')

In [57]:
df_acoes['Sentimento_Geral'] = Sentimento_Geral

In [58]:
Sentimento_Geral = df_acoes['Sentimento_Geral']

In [59]:
Sentimento_Geral

Date
2019-06-03                                NE
2019-06-04                            [[PO]]
2019-06-05                      [[NE], [NE]]
2019-06-06                      [[PO], [NE]]
2019-06-07    [[NG], [NE], [NG], [PO], [PO]]
2019-06-10                            [[NE]]
Name: Sentimento_Geral, dtype: object

In [60]:
for i in range(len(Sentimento_Geral)):
    if type(Sentimento_Geral[i]) == list:
        Sentimento_Geral[i] = list(itertools.chain(*Sentimento_Geral[i]))

C:\Users\Arthur\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
for i in range(len(Sentimento_Geral)):
    lista = []
    for j in range(len(Sentimento_Geral[i])):
        if Sentimento_Geral[i][j] == 'NE':
            lista.append(0)
        if Sentimento_Geral[i][j] == 'NG':
             lista.append(-1)
        if Sentimento_Geral[i][j] == 'PO':
             lista.append(1)
    Sentimento_Geral[i] = lista
                
        

C:\Users\Arthur\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [62]:
for i in range(len(Sentimento_Geral)):
    Sentimento_Geral[i] = sum(Sentimento_Geral[i])

C:\Users\Arthur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
lista = []
for i in range(len(Sentimento_Geral)):
    if Sentimento_Geral[i] > 0:
        lista.append('PO')
    if Sentimento_Geral[i] < 0:
        lista.append('NG')
    if Sentimento_Geral[i] == 0:
        lista.append('NE')


In [64]:
df_acoes['Sentimento_Geral'] = lista

In [65]:
df_acoes['Vari'] = df_acoes['Close'] -  df_acoes['Open']

In [66]:
Variacao = df_acoes['Vari']

In [67]:
Variacao_geral = []
for i in range(len(Variacao)):
    if Variacao[i] > 0.05:
        Variacao_geral.append('Subiu')
    elif Variacao[i] < -0.05:
        Variacao_geral.append('Deceu')
    else:
        Variacao_geral.append('Manteve')

In [68]:
df_acoes['Vari'] = Variacao_geral

In [69]:
df_acoes = df_acoes.drop(columns=['High','Low','Open','Close','Volume','Adj Close'])

In [70]:
df_acoes

,Sentimento_Geral,Vari
Date,,
2019-06-03,NE,Subiu
2019-06-04,PO,Deceu
2019-06-05,NE,Deceu
2019-06-06,PO,Subiu
2019-06-07,NE,Subiu
2019-06-10,NE,Manteve


In [71]:
le = preprocessing.LabelEncoder() #transforma atributos qualitativos em quantitativos
for column in df_acoes.columns:
    if df_acoes[column].dtypes == 'object':
        df_acoes[column] = le.fit_transform(df_acoes[column])

In [72]:
corr_matrix = df_acoes.corr()

In [73]:
print(corr_matrix)

                  Sentimento_Geral      Vari
Sentimento_Geral          1.000000 -0.131306
Vari                     -0.131306  1.000000
